<a href="https://colab.research.google.com/github/JanghyukChoi/ML-Deeplearning-Basic/blob/main/Deep%20Learning/Deep_Learning_ColorGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1) Importing Python Packages for GAN**


In [ ]:
# from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D, Dense, Conv2DTranspose
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images resized_images

In [ ]:
!unzip /content/drive/MyDrive/Bobross/paintings5k.zip -d paintings

Archive:  /content/drive/MyDrive/Bobross/paintings5k.zip
  inflating: paintings/45e1dde296c3328e8fcc3f5f848e38d6c.jpg  
  inflating: paintings/45eb3bf6310071a52b1c6a76868975dcc.jpg  
  inflating: paintings/45ed56a8bdeec60bf33e844c3c273217c.jpg  
  inflating: paintings/45f9aaa4ca278fcda227df3f4659eb15c.jpg  
  inflating: paintings/45f36a6fc1e8e623a221fd5909b51093c.jpg  
  inflating: paintings/45f291f0b53432f1477b1d075f3e1af2c.jpg  
  inflating: paintings/45f560b2c0039bbef71c5df421446f27c.jpg  
  inflating: paintings/000c6828b825f032af6047b46eba2686c.jpg  
  inflating: paintings/000e1fa33ba1ffd07953c3de9898e5e1c.jpg  
  inflating: paintings/00afb8e719aa2ea716a5b6a54c5c55fbc.jpg  
  inflating: paintings/00bd05a5d525f451228196e47d51e243c.jpg  
  inflating: paintings/00c4203c603bf46957ad9030f6712e9ac.jpg  
  inflating: paintings/00c775299a9b11d6a4d310a1464d7493c.jpg  
  inflating: paintings/00ca56f16c0bae52185ea31f95f0484cc.jpg  
  inflating: paintings/00d0e9ea9c51e85b92133b019430adf8c.jpg 

In [ ]:
images_path = "/content/paintings/"

## **Resizing Data to match Neural Network Input**

In [ ]:
import os
# from PIL import Image
import cv2
reshape_size = (128,128)

i = 0
for image in os.listdir(images_path):
  #print(image)
  img = cv2.imread(images_path + image)
  img = cv2.resize(img, reshape_size)
  cv2.imwrite("resized_images/%d.png" % i,img)
  # # print(img.shape)
  i = i+1

## **2) Parameters for Neural Networks & Data**

In [ ]:
img_width = 128
img_height = 128
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0002)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def build_generator():
    model = Sequential()
    #(128/2)/2/2 = 16
    model.add(Dense(256 * 16*16, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((16,16,256)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

    model.summary()

    return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 65536)             6619136   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 65536)             0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 128)      524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 64, 64, 128)      262272    
 ranspose)                                              

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3,3), padding='same', ))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 128, 128, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 128)     147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 256)    

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001

    # Rescale images 0 - 1
    gen_imgs = (gen_imgs + 1) / 2.0

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("currentgeneration.png")
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

In [ ]:
from PIL import Image

def train(epochs, batch_size=32, save_interval=200):

  array = []
  #PUT PATH OF RESIZED IMAGES
  path = "/content/resized_images/"

  for dir in os.listdir(path):
            # print(dir)
    image = Image.open(path + dir)
    data = np.asarray(image)
    array.append(data)

  X_train = np.array(array)
  print(X_train.shape)

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  bat_per_epo = int(X_train.shape[0] / batch_size)
  # X_train = np.expand_dims(X_train, axis=3)
  print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
      #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      #Generate Fake Images
      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      #Train discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0, 1, (batch_size, latent_dim))

      #inverse y label
      g_loss = GAN.train_on_batch(noise, valid)

      print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,j, d_loss[0], 100* d_loss[1], g_loss))

      # if(epoch % save_interval) == 0:
    save_imgs(epoch)


train(10, batch_size=32, save_interval=200)

(3564, 128, 128, 3)
(3564, 128, 128, 3)
******* 0 0 [D loss: 0.723742, acc: 21.88%] [G loss: 0.718688]
******* 0 1 [D loss: 0.306795, acc: 100.00%] [G loss: 0.934898]
******* 0 2 [D loss: 0.207085, acc: 100.00%] [G loss: 1.371298]
******* 0 3 [D loss: 0.100103, acc: 100.00%] [G loss: 2.242311]
******* 0 4 [D loss: 0.107553, acc: 95.31%] [G loss: 2.672549]
******* 0 5 [D loss: 0.027684, acc: 100.00%] [G loss: 3.265657]
******* 0 6 [D loss: 0.014435, acc: 100.00%] [G loss: 3.940396]
******* 0 7 [D loss: 0.007495, acc: 100.00%] [G loss: 4.614142]
******* 0 8 [D loss: 0.004335, acc: 100.00%] [G loss: 5.162984]
******* 0 9 [D loss: 0.003731, acc: 100.00%] [G loss: 5.346205]
******* 0 10 [D loss: 0.005591, acc: 100.00%] [G loss: 5.062161]
******* 0 11 [D loss: 0.040269, acc: 100.00%] [G loss: 4.465070]
******* 0 12 [D loss: 0.181293, acc: 93.75%] [G loss: 4.409723]
******* 0 13 [D loss: 0.124366, acc: 96.88%] [G loss: 6.233131]
******* 0 14 [D loss: 0.006422, acc: 100.00%] [G loss: 9.997399]